<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./30_cams_eac4_retrieve.ipynb"><< 30 - CAMS EAC4 - Retrieve</a><span style="float:right;">

<div class="alert alert-block alert-warning">
<b>LOAD, BROWSE AND VISUALIZE</b></div>

# Copernicus Atmosphere Monitoring Service (CAMS) Global Reanalysis (EAC4) 

The Copernicus Atmopshere Monitoring Service (CAMS) provides consistent and quality-controlled information related to air pollution and health and greenhouse gases. CAMS data consist of `global forecasts and analyses`, `global reanalyses (EAC4)`, `fire emissions` and `greenhouse gas flux inversions`.

This notebooks provides an introduction to the CAMS global reanalysis (EAC4) data. EAC4 (ECMWF Atmospheric Composition Reanalysis 4) is the 4th generation ECMWF global reanalysis of atmospheric composition. Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset. As a trace gas for smoke and fires the variable `Organic Matter Aerosol Optical Depth` is used. Alternatively, you can use `Total Aerosol Optical Depth` or `Total Column Carbon Monoxide` to monitor fires.

CAMS EAC4 data are available in either `GRIB` or `netCDF` format. Get more information in the [CAMS Reanalysis data documentation](https://confluence.ecmwf.int/display/CKB/CAMS%3A+Reanalysis+data+documentation) and at the [Copernicus Atmosphere Data Store](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-global-reanalysis-eac4?tab=overview).

#### Module outline:
* [1 - Load and browse organic matter aerosol optical depth (AOD) at 550nm](#load_browse)
* [2 - Bring longitude coordinates onto a (-180,180) grid](#shift)
* [3 - Retrieve the data variable organic matter AOD at 550nm as xarray DataArray](#data_retrieve)
* [4 - Visualize organic matter aerosol optical depth at 550nm](#visualize)
* [5 - Create a geographical subset for Australia](#subset)

<hr>

#### Load required libraries

In [ ]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc
import pandas as pd

from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.cm import get_cmap
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh


#### Load helper functions

In [ ]:
from ipynb.fs.full.functions import visualize_pcolormesh, generate_geographical_subset

<hr>

### <a id='load_browse'></a>1. Load and browse `EAC4 reanalysis` data

CAMS global reanalysis data is available either in `GRIB` or `netCDF`. The data for this example is available in `netCDF`. 
The code below searches in the directory all the netCDF file and provides the filename of the last one downloaded.

In [ ]:
# get filename of latest .nc downloaded file 
files = [file for file in os.listdir(".") if (file.lower().endswith('.nc'))]
list_nc_file = []

for file in sorted(files,key=os.path.getmtime, reverse=True):
    list_nc_file.append(file)
    
print(f'Selected netCDF file: {list_nc_file[0]}')   

You can use xarray's function `xr.open_dataset()` to open the netCDF file as xarray Dataset.

In [ ]:
file = xr.open_dataset(list_nc_file[0])
file


The data above has three dimensions (`latitude`, `longitude` and `time`) and three data variables:
* `omaod550`: Organic matter Aerosol Optical Depth at 550nm , 
* `aod550`: Total Aerosol Optical Depth at 550nm, and 
* `tcco`: Total Column Carbon Monoxide.

Let us inspect the coordinates of the file more in detail.

Below, you see that the data set consists of 56 time steps, ranging from 25 December 2019 00 UTC to 31 December 21 UTC in a three-hourly timestep.

In [ ]:
file.time

The latitude values have a 0.25 degrees resolution and have a global N-S coverage.

In [ ]:
file.latitude

The longitude values have a 0.25 degrees resolution as well, but are on a [0,360] grid instead of a [-180,180] grid. 

In [ ]:
file.longitude

### <a id='shift'></a>2. Bring longitude coordinates onto a [-180,180] grid

You can assign new values to coordinates in an xarray Dataset. You can do so with the `assign_coords()` function, which you can apply onto an xarray Dataset. With the code below, you shift your longitude grid from [0,360] to [-180,180]. At the end, you sort the longitude values in an ascending order.

In [ ]:
file_assigned = file.assign_coords(longitude=(((file.longitude + 180) % 360) - 180)).sortby('longitude')
file_assigned

A quick check of the longitude coordinates of the new xarray Dataset shows you that the longitude values range now between [-180, 180].

In [ ]:
file_assigned.longitude

### <a id='data_retrieve'></a>3. Retrieve the data variable `organic matter AOD at 550nm` as xarray DataArray

Let us store the data variable `organic matter AOD at 550nm` as xarray DataArray with the name `om_aod`.

In [ ]:
om_aod = file_assigned.omaod550
om_aod

Above, you see that the variable `om_aod` has two attributes, `units` and `long_name`. Let us define variables for those attributes. The variables can be used for visualizing the data.

In [ ]:
long_name = om_aod.long_name
units = om_aod.units

Let us do the same for the coordinates `longitude` and `latitude`.

In [ ]:
latitude = om_aod.latitude
longitude = om_aod.longitude

### <a id='visualize'></a>4. Visualize `organic matter aerosol optical depth at 550nm`

Let us visualize the dataset. You can use the function [visualize_pcolormesh](./functions.ipynb#visualize_pcolormesh), which makes use of matploblib's function `pcolormesh` and the [Cartopy](https://scitools.org.uk/cartopy/docs/latest/) library.

With `?visualize_pcolormesh` you can open the function's docstring to see what keyword arguments are needed to prepare your plot.

In [ ]:
?visualize_pcolormesh

You can make use of the variables we have defined above:
- `units`
- `long_name`
- `latitude`
- `longitude`

Additionally, you can specify the color scale and minimum and maxium data values.

In [ ]:
visualize_pcolormesh(
    om_aod[15,:,:],
    longitude,
    latitude,
    ccrs.PlateCarree(),
    'YlGn',
    units,
    long_name + ' ' + str(om_aod[15,:,:].time.data),
    0, 1.5,
    -180,180,-90,90,
    log=False,
    set_global=True
    )

<br>

### <a id='subset'></a>5. Create a geographical subset for Australia

The map above shows organic matter of Aerosol Optical Depth at 550nm globally. Let us create a geographical subset for Australia, in order to better analyse the Australian wildfires.

For geographical subsetting, you can use the function [generate_geographical_subset](./functions.ipynb#generate_geographical_subset). You can use `?generate_geographical_subset` to open the docstring in order to see the function's keyword arguments.

In [ ]:
?generate_geographical_subset

Define the bounding box information

In [ ]:
latmin = -70
latmax = 10
lonmin = 100
lonmax = 179.5

Now, let us apply the function [generate_geographical_subset](./functions.ipynb#generate_geographcial_subset) to subset the `om_aod` DataArray. Let us call the new DataArray `om_aod_subset`.

In [ ]:
om_aod_subset = generate_geographical_subset(om_aod, latmin, latmax, lonmin, lonmax)
om_aod_subset

Let us visualize the subsetted DataArray again. This time, you set the `set_global` kwarg to `False` and you specify the longitude and latitude bounds specified above.

Additionally - in order to have the time information as part of the title, we add the string of the datetime information to the `long_name` variable: `long_name + ' ' + str(om_aod_subset[50,:,:].time.data)`.

In [ ]:
visualize_pcolormesh(
    om_aod_subset[55,:,:],
    om_aod_subset.longitude,
    om_aod_subset.latitude,
    ccrs.PlateCarree(),
    'YlGn',
    units,
    long_name ,
    0, 2,
    lonmin,lonmax,latmin,latmax,
    log=False,
    set_global=False
    )

<br>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./30_cams_eac4_retrieve.ipynb"><< 30 - CAMS EAC4 - Retrieve</a><span style="float:right;">

<hr>

<p><img src='./img/all_partners_wekeo.png' align='left' alt='Logo EU Copernicus' width='100%'></img></p>